In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


df= pd.read_csv("csv/table_complete_accidents_mobilité_douce_avec_pivot_véhicules.csv", sep=";",dtype=str)

ON SE CONCENTRE SUR PARIS

In [2]:
df=df[df['Département']=="75"]

MANIP DE TYPES POUR POUVOIR UTILISER LES INFOS

In [3]:
df['Latitude']=df['Latitude'].str.replace(",",".")
df['Longitude']=df['Longitude'].str.replace(",",".")
df["Latitude"]=df["Latitude"].astype('float64')
df["Longitude"]=df["Longitude"].astype('float64')

df['Heure']=df['Heure'].str.replace("1899-12-30","")
df['Heure']=df['Heure'].str.replace(":00,000","")
df['Heure']=df['Heure'].str.replace(r'[:]\d{2}',"",regex=True)

df['Année de naissance1']=df['Année de naissance1'].str.replace("-01-01 00:00:00,000","")
df['Année de naissance2']=df['Année de naissance2'].str.replace("-01-01 00:00:00,000","")

In [4]:
# ANALYSE MD vs piéton
catv = ["01 - Bicyclette", "80 - VAE", "50 - EDP à moteur", "60 - EDP sans moteur"]

df = df[df['Catégorie du véhicule1'].isin(catv)]
df = df[df['Catégorie du véhicule2'].isin(catv)]
df=df[df['Catégorie usager2'].isin(["3 - Piéton"])]

In [5]:
df.shape

(1117, 196)

C'EST PARTI

In [6]:
df=df.sort_values('Année')
fig = px.histogram(df,x='Année')
fig.show()

In [7]:
fig = px.histogram(df,x='Catégorie du véhicule1')
fig.show()

In [8]:
df=df.sort_values("Année")
fig = px.density_heatmap(df,x='Longitude',y='Latitude', animation_frame='Année', nbinsx=20,nbinsy=15 )
fig.show()

In [9]:
df2=df

df2=df2.sort_values("Année")

fig = px.density_mapbox(df2,lat="Latitude", lon="Longitude",animation_frame = "Année", radius=11,
                        mapbox_style="open-street-map", zoom=11, height=600)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000

fig.show()

In [10]:
df2=df

df2=df2.sort_values("Heure")

fig = px.density_mapbox(df2,lat="Latitude", lon="Longitude",animation_frame = "Heure", radius=9,
                        animation_group='Latitude',mapbox_style="open-street-map", zoom=11, height=600)
fig.show()

In [11]:

fig = go.Figure()
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],subplot_titles=['Conducteur', 'Piéton'])
fig.add_trace(go.Pie(labels=df['Sexe1'], name="Sexe conducteur"),
              1, 1)
fig.add_trace(go.Pie(labels=df['Sexe2'], name="Sexe piéton"),
              1, 2)
fig.update_layout(
    title_text="Sexe des personnes impliquées (nouvelle mobilité vs piéton)",
    # Add annotations in the center of the donut pies.
    legend=dict(
        title=dict(
            text="Sexe"
        ))
    )
fig.show()

COMPARAISON DES AGES

In [12]:
count_col1 = df['Année de naissance1'].value_counts()
count_col2 = df['Année de naissance2'].value_counts()

# Combine the results into a single DataFrame and add 0's in empty cells
combined_counts = pd.DataFrame({'Column1_Counts': count_col1,'Column2_Counts': count_col2}).fillna(0).astype(int)  
combined_counts.reset_index(inplace=True)

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=combined_counts['index'], y=combined_counts['Column1_Counts'],fill='tozeroy', name="Conducteur-trice",
                         )) # fill down to xaxis
fig.add_trace(go.Scatter(x=combined_counts['index'], y=combined_counts['Column2_Counts'], fill='tozeroy',name="Piéton-ne")) # fill to trace0 y
fig.update_layout(
    height=500,
    width=800,
    title="Comparaison des âges des personnes impliquées (2019-2023)",
    xaxis=dict(
        title=dict(
            text="Année de naissance"
        )
    ),
    yaxis=dict(
        
        title=dict(
            text="Nombre d'accidents"
        )
    ),
    legend=dict(
        yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
        title=dict(
            text="Année de naissance"
        )
    )
)
fig.show()

GRAVITE DES BLESSURES

In [14]:
df=df.sort_values('Année de naissance2')
fig = px.histogram(df,x='Année de naissance2',color='Gravité des blessures2')
fig.update_layout(
    title=dict(
        text="Gravité des blessures pour les piétons"
    )
)
fig.show()

In [15]:
df['Année de naissance1'].value_counts().shape

(69,)

In [22]:
df=df.sort_values(['Année de naissance1','Gravité des blessures1'],ascending=True)
fig = px.histogram(df,x='Année de naissance1', color='Gravité des blessures1', nbins=80)
fig.update_layout(
    title=dict(
        text="Gravité des blessures pour les conducteurs de MD"
    ),
    xaxis = dict(type = 'date'),
    bargap=0.1
)
fig.show()